# San Francisco Employee Compensation from 2013-2022
## By Özge Terzioğlu
## 19 March 2023

#### Story background

Last year, it was reported that [58% of San Francisco's ciy employees don't actually live in San Francisco](https://www.sfchronicle.com/sf/article/remote-work-17510522.php#:~:text=The%20majority%20of%20people%20who,S.F.). Most of these employees commmute from surrounding Bay Area counties and even outside of the Bay Area from areas like Sacramento and San Joaquin county. 

This commute crisis is a reflection of the increasingly unaffordable cost of living in San Francisco and **suggests that City employees salaries are suitable for surrounding Bay Area counties but not for the City itself.**

#### About the dataset

The [dataset](https://data.sfgov.org/City-Management-and-Ethics/Employee-Compensation/88g8-5mnd) in this notebook is from data.sfgov.org. It is a database of salaries and benefits paid to City employees from 2013-2023 (the present). This notebook will also use the [annual Consumer Price Index (CPI)](https://www.bls.gov/regions/west/ca_sanfrancisco_md.htm) to see how City employees' salaries have kept up with inflation. Perhaps another cost of living dataset can be used to sharpen this analysis. 

This database is maintained by the San Francisco Controller's Office and new data is added on a bi-annual basis (for fiscal and calendar years).

According to the [FAQ](https://support.datasf.org/help/employee-compensation-faq) about this database, it's possible for one employee to appear on multiple lines. One should filter `Year Type` column before analyzing data because the salary data is recorded for both fiscal and calendar year. Some rows can have zero dollar amounts for various reasons. Some dollar counts can be in the negative due to adjustments to their pay.

#### Key fields for analysis

- `Year Type`: Fiscal (July-June) or Calendar (January-December)
- `Year`: accounting period of 12 months
- `Job Family`: combines similar jobs into meaningful groups
- `Employee Identifier`: Each distinct number represents one employee. These numbers are randomly assigned. Each time dataset is update, employee identifiers change so comparison across multiple versions of this dataset is not possible. 
- `Salaries`: salaries paid to permanent or temporary City employees
- `Overtime`: money paid to City employees who work over 40 hours per week.
- `Total Compensation`: The sum of all salaries and benefits paid to City employees.

#### Roadbumps

- A few times my visuals were displaying weirdly (year had commas) because I didn't realize that the `Year` in the dataset was an integer and not a string. I changed its datatype and it worked. 
- Some fields had nothing listed for salary but money listed for overtime. Since this is explained by the FAQ I will proceed as normal. 
- The issue of the `Employee Identifier` column is a bit difficult to work with. Each row is one employee, but sometimes employees can appear twice. Do they appear twice with the same ID number? I will have to account for this when I'm trying to get counts of the employees.
- In the data dictionary, it states that total compensation is "the sum of all salaries and benefits paid to City employees." It is unclear if this total number includes overtime pay, as its a separate category from benefits. In theory, it makes sense, but sometimes these dataframes define columns in wacky ways. Best to call and check before proceeding (I did not do this... don't be like me).
- Other than that, the dataset came to me in pretty good shape. There weren't any major red flags or inconsistencies. Most of the cleaning up had to do with the column names or datatypes.

#### Key questions (order of importance)

- Which job family makes the most overtime in pay? 
- Since 2013, how have the salaries of San Francisco government employees measured up to the city's median annual salary? Their overtime pay?
- How much have police salaries changed since 2013?
- Which job family takes home the most in total compensation? The least?

#### Key findings 

- Nursing is the job family with the highest overtime.
- Even though Nursing makes the most in overtime pay, Correction & Detention jobs take home overtime pay as the bulk (and more) or their salary.
- As inflation increased, so did the overtime pay for Correction & Detention and Police Services employees, who make the most overtime pay our of all the job families in San Francisco.  
- Unlike Correction & Detention employees, for Fire Service employees in San Francisco, we see that as inflation increased over the years, overtime pay decreased until 2021, when it dramatically shot up (probably due to a particularly bad wildfire season?). 
- The City's Police Services employees have seen an average increase in total compensation of about $60,000 from 2013-2021.
- Correction & Detention and Police Services employees' overtime pay and the number of employees dramatically increased between 2015 and 2016. 
- Unrepresented Administrative and Management jobs have the highest average total compensation of all San Francisco City employees.

#### Further reading

The San Francisco Chronicle has already done extensive reporting on this dataset. You can check out the stories they found from this dataset below:

- [SF Sheriff's Office has 19 of the city's 20 biggest overtime earners. Here's how much they made](https://www.sfchronicle.com/bayarea/article/SF-sheriff-s-deputies-top-earners-in-overtime-15470686.php)
- [S.F. Mayor Breed’s police overtime push passes first hurdle after fiery debate](https://www.sfchronicle.com/sf/article/sf-mayor-breed-police-overtime-budget-17839701.php)
- [Why San Francisco city workers are making so much money in overtime pay](https://www.sfchronicle.com/sf/article/overtime-pay-san-francisco-17428912.php)
- [200 S.F. city workers made at least $100,000 in overtime pay alone. Here’s a breakdown](https://www.sfchronicle.com/sf/article/overtime-city-workers-pay-17385912.php)

## Grab the Data

### Let's import our tools

Firstly, we'll need to import the necessary modules for reading and writing to the csv file, analyzing the data, modifying the dates to analyze time, and creating visuals in this notebook. 

In [1]:
import csv 
import requests
import pandas as pd
from datetime import datetime as dt
import altair as alt
from vega_datasets import data
import os
from pathlib import Path

### Let's locally read our data into this notebook

In [2]:
#SF employee compensation data
url = "https://www.dropbox.com/s/uyvk2grftra4xvh/Employee_Compensation.csv?dl=1"
comp = pd.read_csv(url)

#SF median annual salary data 2013-2021 (courtesy of U.S. Census Bureau and American Community Survey)
link = "https://www.dropbox.com/s/afmwx4dzd2hbcev/ACS_SF_income_data.csv?dl=1"
city_salary_median = pd.read_csv(link)

Our datafile was too large to locally read so we're reading it via a Dropbox url.

### Edit column names

To make the data analysis go smoothly, let's turn the column names to snakecase in true Pythonista fashion. 

In [47]:
comp.columns = [col.lower().replace(" ", "_") for col in comp.columns]

## Vetting the data

### What does our dataset look like?

In [4]:
comp.head()

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,...,employee_identifier,salaries,overtime,other_salaries,total_salary,retirement,health_and_dental,other_benefits,total_benefits,total_compensation
0,3,1400,1404,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,790.0,"SEIU, Local 1021, Misc",...,37486688,60720.01,0.00,0.00,60720.01,13653.20,14733.76,4904.34,33291.30,94011.31
1,3,9700,9703,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,39646203,91677.00,0.00,0.00,91677.00,17524.20,14733.76,7411.13,39669.09,131346.09
2,3,2900,2918,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,37486043,89106.03,0.00,1540.00,90646.03,17327.20,14733.76,7401.92,39462.88,130108.91
3,3,2900,2918,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,37615772,85581.11,3355.94,337.75,89274.80,16359.16,14151.56,7096.21,37606.93,126881.73
4,3,2900,2905,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,37500212,86457.00,0.00,2090.00,88547.00,16925.97,14733.76,7257.89,38917.62,127464.62


Now let's get a feel for each column.

In [5]:
comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799562 entries, 0 to 799561
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   organization_group_code  799562 non-null  int64  
 1   job_family_code          799562 non-null  object 
 2   job_code                 799562 non-null  object 
 3   year_type                799562 non-null  object 
 4   year                     799562 non-null  int64  
 5   organization_group       799562 non-null  object 
 6   department_code          799560 non-null  object 
 7   department               799560 non-null  object 
 8   union_code               799383 non-null  float64
 9   union                    799383 non-null  object 
 10  job_family               799562 non-null  object 
 11  job                      799557 non-null  object 
 12  employee_identifier      799562 non-null  int64  
 13  salaries                 799562 non-null  float64
 14  over

This is a robust dataset. The columns I will find most useful for my analysis will be year, department, job, overtime, and total compensation. I can use organization_group to make observations based on the jobs or departments with similar functions (i.e. police and firefighters make up public safety). 

It is also notable that for every employee identifier, there is salary and benefit information, but there's a few blanks for department_code, department, and job. We won't look into the missing union values because not every job is protected by a union.

The year being an integer and not a string might cause problems later, let's fix that.

In [6]:
comp = comp.astype({"year":"string"})

### Why do some entries have missing "job" descriptors?

In [7]:
check_col = comp[pd.isnull(comp["job"])]
pd.set_option('display.max_columns', None)
check_col

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,job_family,job,employee_identifier,salaries,overtime,other_salaries,total_salary,retirement,health_and_dental,other_benefits,total_benefits,total_compensation
427,4,0900,2246,Fiscal,2018,Community Health,DPH,Public Health,351.0,"Municipal Exec Assoc, Misc",Management,NaN,37303832,0.00,0.0,13524.71,13524.71,0.00,0.00,1034.64,1034.64,14559.35
42403,4,0900,2246,Calendar,2017,Community Health,DPH,Public Health,351.0,"Municipal Exec Assoc, Misc",Management,NaN,37303832,56887.83,0.0,13524.71,70412.54,10382.10,8388.41,5633.01,24403.52,94816.06
126248,4,0900,2246,Fiscal,2017,Community Health,DPH,Public Health,351.0,"Municipal Exec Assoc, Misc",Management,NaN,37303832,120426.76,0.0,0.00,120426.76,21994.01,17084.16,9237.63,48315.80,168742.56
216671,5,2600,2652,Fiscal,2017,Culture & Recreation,REC,Recreation And Park Commission,250.0,"SEIU, Local 1021, Misc",Dietary & Food,NaN,36938110,6455.33,0.0,0.00,6455.33,0.00,0.00,501.03,501.03,6956.36
245980,5,3300,3320,Fiscal,2017,Culture & Recreation,REC,Recreation And Park Commission,856.0,"Teamsters, Local 856, Multi",Park & Zoo,NaN,37352733,2096.80,0.0,12190.60,14287.40,390.21,383.32,1119.51,1893.04,16180.44


It seems like some of these employees are part-timers. Perhaps this could have something to do with the lack of job title. Some of the employees with missing job entries are full-timers, so this error should be looked into by calling the data expert.

### Are there any gaps in the reported years?

Now I want to find out how many years are accounted for in this dataset and see if there's any gaps in reported years. I also want to get an idea how many city employees' salaries were logged each year over these 9 years. We will visualize this with a quick line chart.

In [8]:
year_count = comp["year"].value_counts()

In [9]:
entries_per_year = year_count.reset_index().set_axis(["year", "employee_count"], axis = 1)

In [10]:
alt.Chart(entries_per_year).mark_line().encode(
    x = "year",
    y = "employee_count"
).properties(
    title = "Number of San Francisco City Employees ",
    width = 600,
    height = 300
)

alt.Chart(...)

If every entry is a unique employee, we can assume that in 2019 the city of San Francisco employed the most people! We can also see that in 2022 there are remarkably less entries than other years. We can chalk this up to the database last being updated February 20, 2023. Some entries are by fiscal year, some by calendar. The 2022 fiscal year ends in June 2023, so this year is incomplete. For this purpose, we will exclude it from our analysis.

Now, let's look at employee_identifier to ensure that there are no duplicates within the same year. The data dictionary tells us that the employee identifier is randomly assigned for the purpose of building the dataset, and that every row is one unique employee. For this reason, I don't think we can compare an employee ID from one year to the next to see how that specific employee's salary evolved. But let's check. 

In [11]:
comp.employee_identifier.nunique()

432571

In [12]:
comp.employee_identifier.info()

<class 'pandas.core.series.Series'>
RangeIndex: 799562 entries, 0 to 799561
Series name: employee_identifier
Non-Null Count   Dtype
--------------   -----
799562 non-null  int64
dtypes: int64(1)
memory usage: 6.1 MB


There are duplicate values in the employee_identifier column, which means that the employee_identifier is probably the same for specific employees from year to year.

## The Analysis

### Which job family takes home the most in total compensation? 

Let's start by filtering our dataset by calendar year so we can include all the data up to 2022 (since 2023 is incomplete as of now).

In [13]:
comp_filtered = comp[comp.year_type == "Calendar"]
comp_filtered = comp_filtered[comp_filtered.year <= "2022"]
comp_filtered

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,job_family,job,employee_identifier,salaries,overtime,other_salaries,total_salary,retirement,health_and_dental,other_benefits,total_benefits,total_compensation
1901,1,8300,8304,Calendar,2018,Public Protection,SHF,Sheriff,498.0,Deputy Sheriffs' Assoc (DSA),Correction & Detention,Deputy Sheriff,38965911,108881.24,70891.25,9405.28,189177.77,38481.12,15155.49,3170.21,56806.82,245984.59
1902,1,8300,8310,Calendar,2018,Public Protection,SHF,Sheriff,499.0,Sheriffs' Mgrs and Supv (MSA),Correction & Detention,Sheriff's Lieutenant,38055928,149283.39,4781.93,21881.75,175947.07,51573.88,16766.45,2884.95,71225.28,247172.35
1903,1,7300,7334,Calendar,2018,Public Protection,SHF,Sheriff,39.0,"Stationary Engineers, Local 39",Journeyman Trade,Stationary Engineer,37466414,98020.24,10795.36,22127.70,130943.30,19430.96,15484.04,10530.25,45445.25,176388.55
1904,1,8300,8304,Calendar,2018,Public Protection,SHF,Sheriff,498.0,Deputy Sheriffs' Assoc (DSA),Correction & Detention,Deputy Sheriff,37462201,109842.03,7335.48,13619.24,130796.75,37099.30,15287.20,2172.04,54558.54,185355.29
1905,1,8300,8306,Calendar,2018,Public Protection,SHF,Sheriff,498.0,Deputy Sheriffs' Assoc (DSA),Correction & Detention,Senior Deputy Sheriff,37765796,0.00,0.00,0.00,0.00,0.00,19802.79,0.00,19802.79,19802.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799552,4,2900,2908,Calendar,2016,Community Health,DPH,DPH Public Health,535.0,"SEIU - Human Services, Local 1021",Human Services,Hospital Eligiblity Worker,53366,79803.21,0.00,1085.00,80888.21,14884.29,13297.76,6638.80,34820.85,115709.06
799553,4,2900,2908,Calendar,2015,Community Health,DPH,DPH Public Health,535.0,"SEIU - Human Services, Local 1021",Human Services,Hospital Eligiblity Worker,53366,76066.19,0.00,1165.00,77231.19,15885.96,12262.32,6343.98,34492.26,111723.45
799554,4,2900,2908,Calendar,2014,Community Health,DPH,DPH Public Health,535.0,"SEIU - Human Services, Local 1021",Human Services,Hospital Eligiblity Worker,53366,72606.17,0.00,205.00,72811.17,15889.52,12705.72,5932.61,34527.85,107339.02
799555,4,2900,2908,Calendar,2013,Community Health,DPH,DPH Public Health,535.0,"SEIU - Human Services, Local 1021",Human Services,Hospital Eligiblity Worker,53366,71221.94,0.00,125.00,71346.94,13995.54,12262.11,5818.46,32076.11,103423.05


Let's peek at what we're working with.

In [14]:
comp_filtered["job_family"].head()

1901    Correction & Detention
1902    Correction & Detention
1903          Journeyman Trade
1904    Correction & Detention
1905    Correction & Detention
Name: job_family, dtype: object

In [15]:
groupby_money = comp_filtered.groupby("job_family")["total_compensation"].mean().reset_index().sort_values(by = "total_compensation", ascending = False)
groupby_money

,job_family,total_compensation
0,Administrative & Mgmt (Unrep),342719.270696
57,Unassigned,215158.259848
18,Fire Services,188585.754351
3,Administrative-Labor & Trades,185316.959125
12,Construction Project Mgmt,183610.461219
29,Management,181413.661532
2,Administrative-DPW/PUC,180072.672810
31,Medical & Dental,175658.260773
37,Police Services,170986.814782
26,Legal & Court,164537.727793


In [16]:
alt.Chart(groupby_money.head()).mark_bar().encode(
    x = "job_family",
    y = "total_compensation",
    color = "total_compensation"
    ).properties(title = "Average Total Compensation of San Francisco Employees (2013-2022)", height = 400, width = 600)

alt.Chart(...)

**Finding:** Unrepresented Administrative and Management jobs have the highest average total compensation of all San Francisco City employees. Does this job family have the highest number of employees as well? This would be important to look into. 

### How much have SF police salaries changed since 2013?

In [17]:
new_df = comp_filtered[["job_family","year","total_compensation"]].copy()
police_filter = new_df[new_df["job_family"].isin(["Police Services"])]
comp_grouped = police_filter.groupby(["job_family","year"])
police_filter = comp_grouped.mean()
police_filter = police_filter.reset_index()
police_filter

,job_family,year,total_compensation
0,Police Services,2013,147383.045348
1,Police Services,2014,154129.281045
2,Police Services,2015,151564.139183
3,Police Services,2016,153279.984178
4,Police Services,2017,173193.247215
5,Police Services,2018,171312.301795
6,Police Services,2019,194701.645736
7,Police Services,2020,199301.083307
8,Police Services,2021,205534.519198


In [18]:
alt.Chart(police_filter).mark_line().encode(
    x = "year",
    y = "total_compensation"
).properties(title = "San Francisco Police Services Employees' Total Compensation", height = 400, width = 600)

alt.Chart(...)

**Finding:** The City's Police Services employees have seen an average increase in total compensation of about $60,000 from 2013-2021.

### Which job family makes the most overtime in San Francisco? 

We want to look at the "overtime" column when doing our analysis, as that gives a picture of the full salary and benefits received by each position. The value of "overtime" is in dollars.

In [19]:
job_fam_overtime = comp_filtered.groupby("job_family").overtime.count().reset_index().sort_values("overtime", ascending = False)
job_fam_overtime

,job_family,overtime
33,Nursing,40042
54,Street Transit,32019
37,Police Services,23195
24,Journeyman Trade,22203
22,Human Services,19842
8,"Clerical, Secretarial & Steno",16517
18,Fire Services,15818
46,Public Service Aide,15814
21,Housekeeping & Laundry,12491
7,"Budget, Admn & Stats Analysis",11766


In [20]:
alt.Chart(job_fam_overtime.head()).mark_bar().encode(
    x = "job_family",
    y = "overtime",
    color = "overtime"
).properties(
    title = "Overtime Compensation by Job Family",
    width = 600,
    height = 300
)

alt.Chart(...)

**Finding:** Ah, so Nursing is the job family with the highest overtime. It's interesting that nursing makes over double the overtime that police services does. This would be interesting to compare with other comparable cities in the USA. 

Now let's see what's up with individual job titles.

In [21]:
job_overtime = comp_filtered.groupby("job").overtime.count().reset_index().sort_values("overtime", ascending = False)
job_overtime

,job,overtime
1269,Transit Operator,24177
1130,Special Nurse,14203
983,Registered Nurse,12586
527,Firefighter,7782
374,Custodian,7611
...,...,...
1242,"Track Maint Supt, Muni Railway",1
232,Cfdntal Sctry To Distric Atty,1
1132,Specialist in Aging 2,1
546,General Plumber Apprentice I,1


Since job titles in the "job" column vary more in their syntax than job families (e.g. "Speical Assistant 1, Special Assistant 2"), I will focus my analysis on the job_families column. 

### How much overtime pay makes up City employees' total compensation? 

Let's have a look at the total_compensation column to see if we group job_family correctly in this aspect.

In [22]:
total_comp = comp_filtered.groupby("job_family").total_compensation.sum().reset_index().sort_values("total_compensation", ascending = False)
total_comp

,job_family,total_compensation
33,Nursing,4.475530e+09
37,Police Services,3.966039e+09
54,Street Transit,3.471060e+09
24,Journeyman Trade,2.997722e+09
18,Fire Services,2.983049e+09
29,Management,2.126350e+09
22,Human Services,2.110360e+09
40,Professional Engineering,1.615213e+09
13,Correction & Detention,1.604830e+09
23,Information Systems,1.420070e+09


Looks great. Now we'll calculate how much overtime makes up the total compensation paid to employees of each job_family. 

In [23]:
overtime_num = comp_filtered.groupby("job_family").overtime.sum()

In [24]:
total_salary = comp_filtered.groupby("job_family").total_compensation.sum()

In [25]:
pct_of_total = (overtime_num / total_salary) * 1000
pct_of_total = pct_of_total.reset_index().rename(columns = {0:"pct_overtime_pay"})
pct_of_total = pct_of_total.sort_values("pct_overtime_pay", ascending = False)

In [26]:
alt.Chart(pct_of_total.head()).mark_bar().encode(
    x = "job_family",
    y = "pct_overtime_pay",
    color = "pct_overtime_pay"
).properties(
    title = "Percentage of Overtime Pay in San Francisco City Employees' Total Salaries",
    width = 600,
    height = 300
)

alt.Chart(...)

**Finding:** Even though Nursing makes the most in overtime pay, Correction & Detention jobs take home overtime pay as the bulk (and more) or their salary.

Huh, some city employees' overtime pay makes up more than their total compensation. I wonder, then, if it is true that the overtime pay is not included in the total compensation value?

Just because I'm curious let's see the job families who don't rely on overtime pay to make up their salaries.

In [27]:
alt.Chart(pct_of_total.tail()).mark_bar().encode(
    x = "job_family",
    y = "pct_overtime_pay",
    color = "pct_overtime_pay"
).properties(
    title = "Percentage of Overtime Pay in San Francisco City Employees' Total Salaries",
    width = 600,
    height = 300
)

alt.Chart(...)

### How do Correction & Detention city employees' overtime pay measure up with the city's median annual income from 2013-2021? 

In [28]:
city_salary_median.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   year                     9 non-null      int64
 1   median_household_income  9 non-null      int64
dtypes: int64(2)
memory usage: 272.0 bytes


Let's change the year data type to a string.

In [29]:
city_salary_median = city_salary_median.astype({"year":"string"})

In [30]:
city_salary_median.head()

,year,median_household_income
0,2013,77485
1,2014,85070
2,2015,92094
3,2016,103801
4,2017,110816


In order to compare how salaries from different job families compare to the city's median annual salary, we need to organize the data we want to work with from our dataset that excludes 2022. 

Below, we are grouping by our desired fields: `job_family`, `year`, and `overtime`. We want to aggregate this data by the sum of overtime pay for each `job_family` per `year`. 

This work will also allow us to easily visualize our findings. 

First, we'll get the overtime pay for Correction & Detention and Police Services.

In [31]:
employ_id = comp_filtered.drop_duplicates(subset = "employee_identifier", keep = "first", inplace = False, ignore_index = False)
df = employ_id[["job_family", "year", "overtime", "employee_identifier"]].copy()
employ_filtered = df[df["job_family"].isin(["Correction & Detention", "Police Services"])]
employ_filtered = employ_filtered.groupby(["job_family", "year"])
employ_filtered = employ_filtered["overtime"].mean().reset_index()
employ_filtered

,job_family,year,overtime
0,Correction & Detention,2013,768.721231
1,Correction & Detention,2014,1200.763636
2,Correction & Detention,2015,1672.378596
3,Correction & Detention,2016,20162.381472
4,Correction & Detention,2017,28079.744418
5,Correction & Detention,2018,27958.587031
6,Correction & Detention,2019,28681.878009
7,Correction & Detention,2020,31866.419323
8,Correction & Detention,2021,29550.749386
9,Police Services,2013,3488.308634


Now we'll grab the number of employees receiving this overtime pay.

In [32]:
employ_id_2 = comp_filtered.drop_duplicates(subset = "employee_identifier", keep = "first", inplace = False, ignore_index = False)
df_2 = employ_id_2[["job_family", "year", "overtime", "employee_identifier"]].copy()
employ_filtered_2 = df_2[df_2["job_family"].isin(["Correction & Detention", "Police Services"])]
employ_filtered_2 = employ_filtered_2.groupby(["job_family", "year"])
employ_filtered_2 = employ_filtered_2["employee_identifier"].count().reset_index()
employ_filtered_2

,job_family,year,employee_identifier
0,Correction & Detention,2013,65
1,Correction & Detention,2014,44
2,Correction & Detention,2015,57
3,Correction & Detention,2016,1114
4,Correction & Detention,2017,1125
5,Correction & Detention,2018,1108
6,Correction & Detention,2019,1125
7,Correction & Detention,2020,1063
8,Correction & Detention,2021,1042
9,Police Services,2013,183


Now we'll merge the two together.

In [33]:
employ_count_ot = pd.merge(employ_filtered, employ_filtered_2, on = ["year", "job_family"])
employ_count_ot

,job_family,year,overtime,employee_identifier
0,Correction & Detention,2013,768.721231,65
1,Correction & Detention,2014,1200.763636,44
2,Correction & Detention,2015,1672.378596,57
3,Correction & Detention,2016,20162.381472,1114
4,Correction & Detention,2017,28079.744418,1125
5,Correction & Detention,2018,27958.587031,1108
6,Correction & Detention,2019,28681.878009,1125
7,Correction & Detention,2020,31866.419323,1063
8,Correction & Detention,2021,29550.749386,1042
9,Police Services,2013,3488.308634,183


In [34]:
alt.Chart(employ_count_ot).mark_bar(color = "green").encode(
    x = alt.X("year", title = "Year"),
    xOffset = "job_family",
    y = alt.Y("overtime", title = "Average overtime pay"),
    color = alt.Color("employee_identifier", title = "Number of employees"),
    tooltip = [alt.Tooltip("job_family", title = "Job family"), 
               alt.Tooltip("year", title = "Year"), 
               alt.Tooltip("overtime", title = "Average overtime pay (in dollars)"), 
               alt.Tooltip("employee_identifier", title = "Number of employees")]
).interactive().properties(
    title = "Number of City Employess from Highest Overtime Makers in San Francisco", 
    height = 400, 
    width = 600)

alt.Chart(...)

**Finding:** Correction & Detention and Police Services employees' overtime pay and the number of employees dramatically increased between 2015 and 2016. 

### Correction & Detention leads all other categories with overtime pay. Let's do a deep dive on them.

In [35]:
df = comp_filtered[["job_family","year","overtime"]].copy()
filtered_job = df[df["job_family"] == "Correction & Detention"]
ot_grouped = filtered_job.groupby(["job_family","year"])
filtered_job = ot_grouped.mean()
filtered_job = filtered_job.reset_index()
filtered_job

,job_family,year,overtime
0,Correction & Detention,2013,11092.338276
1,Correction & Detention,2014,11916.743213
2,Correction & Detention,2015,15335.324575
3,Correction & Detention,2016,19371.699003
4,Correction & Detention,2017,27620.892938
5,Correction & Detention,2018,27495.888009
6,Correction & Detention,2019,27989.757409
7,Correction & Detention,2020,31781.909010
8,Correction & Detention,2021,29522.416932


We'll rearrange our dataframe to make visualization easier and also merge our grouped job overtime dataframe with San Francisco's annual median salary in order to compare the two. 

In [36]:
most_overtime = filtered_job.pivot(index = "year", columns = "job_family", values = "overtime").reset_index()
most_overtime

job_family,year,Correction & Detention
0,2013,11092.338276
1,2014,11916.743213
2,2015,15335.324575
3,2016,19371.699003
4,2017,27620.892938
5,2018,27495.888009
6,2019,27989.757409
7,2020,31781.909010
8,2021,29522.416932


Here, we're merging our median income dataset and the above overtime compensation data into one dataframe so we can create visuals.

In [37]:
most_overtime = pd.merge(most_overtime, city_salary_median, on = "year")
most_overtime

,year,Correction & Detention,median_household_income
0,2013,11092.338276,77485
1,2014,11916.743213,85070
2,2015,15335.324575,92094
3,2016,19371.699003,103801
4,2017,27620.892938,110816
5,2018,27495.888009,112376
6,2019,27989.757409,123859
7,2020,31781.909010,119136
8,2021,29522.416932,121826


Now we'll add a column with the ratio of Correction & Detention employees' overtime pay with the city's median annual total salary.

In [38]:
most_overtime["pct_ot"] = (most_overtime["Correction & Detention"] / most_overtime["median_household_income"]) * 100
most_overtime

,year,Correction & Detention,median_household_income,pct_ot
0,2013,11092.338276,77485,14.315465
1,2014,11916.743213,85070,14.008162
2,2015,15335.324575,92094,16.651817
3,2016,19371.699003,103801,18.662343
4,2017,27620.892938,110816,24.925004
5,2018,27495.888009,112376,24.467758
6,2019,27989.757409,123859,22.598081
7,2020,31781.909010,119136,26.676999
8,2021,29522.416932,121826,24.233265


In [39]:
alt.Chart(most_overtime).mark_bar().encode(
    x = "year",
    y = alt.Y("median_household_income", title = "SF Annual Median Household Income"),
    color = alt.Color("pct_ot", title = "% of overtime pay to median income")
).properties(title = "Average Overtime Pay for Correction & Detention Employees in SF Relative to City's Median Salary", height = 500, width = 600)

alt.Chart(...)

**Finding:** As the annual median income for the city increased, so did the overtime pay for Correction & Detention employees, who make the most overtime pay our of all the job families in San Francisco. Correction & Detention employees made the most overtime pay relative to the city's annual median income in 2020. 

## How have San Francisco's least paid city employees' salaries compare to the city's median annual average salary?

Now we'll do the same comparison for total compensation of the job families that make the least total compensation. 

In [40]:
total_comp_by_year = comp_filtered[["job_family","year","total_compensation"]].copy()
totalcomp_groupby = total_comp_by_year.groupby(["job_family","year"])
total_comp_by_year = totalcomp_groupby.mean()
total_comp_by_year

total_compensation
job_family                    year                    
Administrative & Mgmt (Unrep) 2013       183028.866000
                              2014       204399.718571
                              2015       365484.100000
                              2016       254162.657778
                              2017       375380.242857
...                                                ...
Untitled                      2017        96947.166510
                              2018        92853.247656
                              2019        95076.750635
                              2020       102607.374868
                              2021        99424.874824

[498 rows x 1 columns]

Time to check the job families who make the least in total compensation (this dataframe was sorted in descending order earlier).

In [41]:
total_comp.tail()

,job_family,total_compensation
15,Emergency Coordination,15141212.27
57,Unassigned,14200445.15
16,Emergency Services,7369409.74
1,Administrative Secretarial,7119868.08
50,SF Redevelopment Agency,4141315.70


To answer our question, then, we'll focus on Emergency Services, Administrative Secretarial, and SF Redevelopment Agency.

In [42]:
df = comp_filtered[["job_family","year","total_compensation"]].copy()
total_comp_job = df[df["job_family"].isin(["Emergency Services", "Administrative Secretarial", "SF Redevelopment Agency"])]
tc_grouped = total_comp_job.groupby(["job_family","year"])
total_comp_job = tc_grouped.mean()
total_comp_job = total_comp_job.reset_index()
total_comp_job

,job_family,year,total_compensation
0,Administrative Secretarial,2013,100496.756364
1,Administrative Secretarial,2014,97070.213000
2,Administrative Secretarial,2015,96309.252000
3,Administrative Secretarial,2016,101450.472222
4,Administrative Secretarial,2017,112671.183333
5,Administrative Secretarial,2018,136035.294000
6,Administrative Secretarial,2019,138807.986000
7,Administrative Secretarial,2020,90752.373333
8,Administrative Secretarial,2021,143199.280000
9,Emergency Services,2013,75639.117333


Now we'll merge this dataframe with our SF annual median income dataframe to see how they compare to each other.

In [43]:
compare_total_comp = pd.merge(total_comp_job, city_salary_median, on = "year")
compare_total_comp

,job_family,year,total_compensation,median_household_income
0,Administrative Secretarial,2013,100496.756364,77485
1,Emergency Services,2013,75639.117333,77485
2,Administrative Secretarial,2014,97070.213000,85070
3,Emergency Services,2014,96006.045789,85070
4,Administrative Secretarial,2015,96309.252000,92094
5,Emergency Services,2015,91658.892174,92094
6,SF Redevelopment Agency,2015,85130.651739,92094
7,Administrative Secretarial,2016,101450.472222,103801
8,Emergency Services,2016,109645.409048,103801
9,SF Redevelopment Agency,2016,83973.488846,103801


In [46]:
base = alt.Chart(compare_total_comp).encode(
    alt.X("year", axis = alt.Axis(title = None))
)

line_1 = base.mark_circle(color = "#57A44C").encode(
    alt.Y("median_household_income",
          axis = alt.Axis(title = "SF Annual Median Income", titleColor = "#57A44C")),
    alt.Y2("total_compensation")
)

line = base.mark_circle(stroke = "#5276A7", interpolate = "monotone").encode(
    alt.Y("total_compensation",
          axis = alt.Axis(title = "City Employees' Total Income", titleColor = "#5276A7"))
)

alt.layer(line_1, line).resolve_scale(
    y = "independent"
).properties(title = "San Francisco's Lowest Paid Jobs Relative to City's Annual Median Income", width = 500, height = 300)

alt.LayerChart(...)

**Finding:** San Francisco's lowest paid employees tend to be compensated below the median annual income of the city's residents. 